In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("Resourses/stockdata.csv", index_col='Index')
missed_tickers = pd.read_csv("Resourses/missedtickers.csv")

In [7]:
missed_tickers

,date,ticker,open,high,low,close,volume,afterHours,preMarket
0,2023-09-01,COR,176.52,177.080,174.220,174.96,1585857.0,174.96,175.98
1,2023-10-02,COR,179.97,180.870,178.210,179.08,1202697.0,179.08,179.97
2,2023-11-01,COR,185.15,188.725,184.950,187.23,1587960.0,187.23,185.15
3,2023-12-01,COR,203.49,205.830,202.470,204.09,1254654.0,204.09,203.49
4,2023-07-03,WKC,20.63,20.870,20.490,20.53,185853.0,20.52,20.63
5,2023-08-01,WKC,22.71,22.790,22.190,22.62,729826.0,22.62,22.71
6,2023-09-01,WKC,22.15,22.520,22.140,22.50,338134.0,22.50,22.15
7,2023-10-02,WKC,22.47,22.470,21.895,22.05,754560.0,22.05,22.47
8,2023-11-01,WKC,18.50,18.760,18.290,18.73,535025.0,18.73,18.50
9,2023-12-01,WKC,20.91,21.600,20.870,21.50,390846.0,21.50,20.91


In [8]:
# Step 2: Replace 'ABC' with 'COR'
df = df.replace('ABC', 'COR')

# Step 3: Replace any cell containing 'INT' with 'WKC'
def replace_INT_with_WKC(cell):
    if 'INT' in str(cell):
        return str(cell).replace('INT', 'WKC')
    else:
        return cell

df = df.applymap(replace_INT_with_WKC)


In [11]:
ticker_column_name = df.columns[1]

# Sort both DataFrames by ticker (for efficiency) and then by date within each ticker
df = df.sort_values(by=[ticker_column_name, 'date'])
missed_tickers = missed_tickers.sort_values(by=[ticker_column_name, 'date'])

# Initialize a list to hold the rows of the new DataFrame
new_rows = []

# Iterate through each unique ticker in missed_tickers
for ticker in missed_tickers[ticker_column_name].unique():
    # Extract rows for the current ticker from both DataFrames
    df_ticker_rows = df[df[ticker_column_name] == ticker]
    missed_ticker_rows = missed_tickers[missed_tickers[ticker_column_name] == ticker]
    
    if not df_ticker_rows.empty:
        # Find the index of the last occurrence of the current ticker in the main DataFrame
        last_index = df_ticker_rows.index[-1]
        
        # Split the main DataFrame into before and after the insertion point
        df_before = df.loc[:last_index]
        df_after = df.loc[last_index + 1:]
        
        # Append the missed_ticker_rows to df_before
        df_before = pd.concat([df_before, missed_ticker_rows])
        
        # Reassemble the DataFrame
        df = pd.concat([df_before, df_after])
    else:
        # If the ticker isn't found in df, just append missed_ticker_rows to the end
        df = pd.concat([df, missed_ticker_rows])

# Reset the index of the final DataFrame
df = df.reset_index(drop=True)




In [13]:
df.to_csv("Resourses/stockdata.csv", index=False)